In [44]:
import os
import base64
import requests
from requests import post, get
import json
from tqdm import tqdm
import pandas as pd
import time

In [45]:
def get_token():
    client_id = '7727b6a6602f4dcf99d337ee11f0983e'
    client_secret = '6b4958253cb544aaae1b0bf4858726ee'
    auth_string = client_id + ":" + client_secret
    auth_bytes = auth_string.encode("utf-8")
    auth_base64 = str(base64.b64encode(auth_bytes), "utf-8")

    url = "https://accounts.spotify.com/api/token"
    headers = {
        "Authorization": "Basic " + auth_base64,
        "Content-Type": "application/x-www-form-urlencoded"
    }
    data = {"grant_type": "client_credentials"}

    response = requests.post(url, headers=headers, data=data)
    json_result = json.loads(response.content)

    if "access_token" in json_result:
        token = json_result["access_token"]
        return token
    else:
        print("Error: Failed to get access token")
        print(json_result)
        return None

def get_auth_header(token):
    return {"Authorization": "Bearer " + token}


def get_artist_id(token, artist_name):
    url = "https://api.spotify.com/v1/search"
    headers = get_auth_header(token)
    query = f"?q={artist_name}&type=artist&limit=1"

    query_url = url + query
    response = requests.get(query_url, headers=headers)
    json_result = json.loads(response.content)

    if "artists" in json_result and "items" in json_result["artists"] and len(json_result["artists"]["items"]) > 0:
        artist = json_result["artists"]["items"][0]
        artist_id = artist.get("id", None)
        return artist_id
    else:
        return None

In [46]:
df = pd.read_parquet("filtered_artists_with_infos.parquet")
df.reset_index(drop=True, inplace=True)
df

,Artist,Spotify Name,Genres,Popularity
0,Motörhead,Motörhead,"[album rock, hard rock, metal, rock, speed metal]",68.0
1,Color Me Badd,Color Me Badd,"[boy band, contemporary r&b, new jack swing, r...",53.0
2,Trumans Water,Trumans Water,[noise rock],7.0
3,Cliff Eberhardt,Cliff Eberhardt,[contemporary folk],10.0
4,The Waitresses,The Waitresses,[new wave pop],40.0
...,...,...,...,...
10111,Wumpscut,:Wumpscut:,"[aggrotech, dark wave, ebm, electro-industrial...",37.0
10112,54-40,54-40,"[canadian indie, canadian rock]",38.0
10113,10_000 Maniacs,"10,000 Maniacs","[folk, lilith, mellow gold, new wave pop, pop ...",53.0
10114,...And You Will Know Us by the Trail of Dead,...And You Will Know Us by the Trail of Dead,"[alternative rock, double drumming, indie rock...",41.0


In [47]:
token = get_token()

df["Spotify ID"] = None

with tqdm(total=len(df)) as pbar:
    for i, artist_name in enumerate(df["Spotify Name"]):
        pbar.set_description(f"Processing artist {i+1} of {len(df)}: {artist_name}")
        if token:
            spotify_id = get_artist_id(token, artist_name)
            if spotify_id is not None:
                df.at[i, "Spotify ID"] = spotify_id
        pbar.update(1)
        time.sleep(0.1)

df


Processing artist 10116 of 10116: 'Wild' Bill Davison: 100%|██████████| 10116/10116 [36:29<00:00,  4.62it/s]                                        


,Artist,Spotify Name,Genres,Popularity,Spotify ID
0,Motörhead,Motörhead,"[album rock, hard rock, metal, rock, speed metal]",68.0,1DFr97A9HnbV3SKTJFu62M
1,Color Me Badd,Color Me Badd,"[boy band, contemporary r&b, new jack swing, r...",53.0,1QtIfAa6y7w2JhxYJhYeUG
2,Trumans Water,Trumans Water,[noise rock],7.0,3GQRTA9DtNlGbQqu8ST1Xx
3,Cliff Eberhardt,Cliff Eberhardt,[contemporary folk],10.0,1LpAlKD69cCn6ygqov2bGs
4,The Waitresses,The Waitresses,[new wave pop],40.0,57AKOY1SF1062tuVzgAPv8
...,...,...,...,...,...
10111,Wumpscut,:Wumpscut:,"[aggrotech, dark wave, ebm, electro-industrial...",37.0,3TTbkikkCAbUg2j5i9UrTV
10112,54-40,54-40,"[canadian indie, canadian rock]",38.0,1Y4l4YawNAZndGBSrZxb3N
10113,10_000 Maniacs,"10,000 Maniacs","[folk, lilith, mellow gold, new wave pop, pop ...",53.0,0MBIKH9DjtBkv8O3nS6szj
10114,...And You Will Know Us by the Trail of Dead,...And You Will Know Us by the Trail of Dead,"[alternative rock, double drumming, indie rock...",41.0,6wBUn8gMP85n8dPu6LoUcF


In [48]:
df.to_parquet("filtered_artists_with_infos_with_id.parquet")